![training](assets/training-basics/puppy-training.jpg)

(image: amazon)

# Training basics

The practice of training machine learning models
- Loss functions
- Gradient descent
- Overfitting, underfitting, bias, variance
- Regularization
- Cross-validation

Objective: a model that trains fast and performs well